In [1]:
import torch.nn    as nn
import torch.optim as optim
import numpy       as np
import torch
import json
import os

from libraries.model      import nGCNN, eGCNN, diffusion_step, add_features_to_graph, get_graph_losses
from libraries.dataset    import standardize_dataset
from libraries.graph      import graph_POSCAR_encoding
from torch.utils.data     import random_split
from torch_geometric.data import Data

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Based on adding and removing noise to materials
# the models is able to learn hidded patterns
# It can be trained regarding some target property

In [3]:
# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_PT'

# Define folder in which all data will be stored
target_folder    = 'models/GM_BiSI'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

In [4]:
# Define target value to look for
#seeked_target = ##

# Machine-learning parameters
n_epochs      = 300
batch_size    = 32
learning_rate = 0.0001

# Ratios for dividing training data
test_ratio = 0.0

# Number of diffusing and denoising steps
n_t_steps = 1

# Decay of parameter alpha
noise_contribution = 0.15
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Create and save as a dictionary
model_parameters = {
    'n_epochs':           n_epochs,
    'batch_size':         batch_size,
    'learning_rate':      learning_rate,
    'test_ratio':         test_ratio,
    'n_t_steps':          n_t_steps,
    'noise_contribution': noise_contribution,
    'dropout_node':       dropout_node,
    'dropout_edge':       dropout_edge
}

# Write the dictionary to the file in JSON format
with open(f'{target_folder}/model_parameters.json', 'w') as json_file:
    json.dump(model_parameters, json_file)

# Generation of graph database for training

Load the datasets, already standardized if possible.

In [6]:
labels_name                 = f'{target_folder}/labels.pt'
dataset_name                = f'{target_folder}/dataset.pt'
dataset_name_std            = f'{target_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_dataset_parameters.json'

if os.path.exists(dataset_name_std) and os.path.exists(dataset_parameters_name_std) and os.path.exists(labels_name):
    # Load the standardized dataset, with corresponding labels and parameters
    dataset = torch.load(dataset_name_std)
    labels  = torch.load(labels_name)
    
    # Load the data from the JSON file
    with open(dataset_parameters_name_std, 'r') as json_file:
        numpy_dict = json.load(json_file)

    # Convert NumPy arrays back to PyTorch tensors
    dataset_parameters = {key: torch.tensor(value) for key, value in numpy_dict.items()}

elif os.path.exists(dataset_name) and os.path.exists(labels_name):
    # Load the raw dataset, with corresponding labels, and standardize it
    dataset = torch.load(dataset_name)
    labels  = torch.load(labels_name)
    
    # Standardize dataset
    dataset, dataset_parameters = standardize_dataset(dataset)
    
    # Save standardized dataset
    torch.save(dataset, dataset_name_std)
    
    # Convert torch tensors to numpy arrays
    numpy_dict = {key: value.cpu().numpy().tolist() for key, value in dataset_parameters.items()}

    # Dump the dictionary with numpy arrays to a JSON file
    with open(parameters_name_std, 'w') as json_file:
        json.dump(numpy_dict, json_file)

else:
    # Generate the raw dataset from scratch, and standardize it
    
    # Read all mateials within the database
    materials = os.listdir(DB_path)
    
    dataset = []
    labels  = []
    for material in materials:
        try:
            # Try to read the polyforms
            polymorfs = os.listdir(f'{DB_path}/{material}')
        except:
            continue
        
        print(material)
        for polymorf in polymorfs:
            # Path to folder containing the POSCAR
            path_to_POSCAR = f'{DB_path}/{material}/{polymorf}'
            
            # Check that the folder is valid
            if os.path.exists(f'{path_to_POSCAR}/POSCAR'):
                print(f'\t{polymorf}')

                # Extract parameters from POSCAR
                cell, composition, concentration, positions = MPL.information_from_VASPfile(path_to_POSCAR,
                                                                                            'POSCAR')

                # Generate POSCAR covering the box
                try:
                    nodes, edges, attributes, _, _, _ = graph_POSCAR_encoding(cell,
                                                                                  composition,
                                                                                  concentration,
                                                                                  positions,
                                                                                  L)
                except:
                    print(f'Error: {material} {polymorf} not loaded')
                    continue

                # Load ground state energy per atom
                gs_energy = float(np.loadtxt(f'{path_to_POSCAR}/EPA'))

                # Construct temporal graph structure
                graph = Data(x=nodes,
                             edge_index=edges,
                             edge_attr=attributes,
                             y=torch.tensor([[gs_energy]], dtype=torch.float)
                            )

                # Append to dataset and labels
                dataset.append(graph)
                labels.append(f'{material}-{polymorf}')
    
    # Standardize dataset
    dataset, dataset_parameters = standardize_dataset(dataset)
    
    # Save standardized dataset
    torch.save(dataset, dataset_name_std)
    torch.save(labels,  labels_name)
    
    # Convert torch tensors to numpy arrays
    numpy_dict = {key: value.cpu().numpy().tolist() for key, value in dataset_parameters.items()}

    # Dump the dictionary with numpy arrays to a JSON file
    with open(dataset_parameters_name_std, 'w') as json_file:
        json.dump(numpy_dict, json_file)

# Definition of train-test datasets

In [7]:
# torch.manual_seed(12345)

# Define the sizes of the train and test sets
test_size  = int(test_ratio * len(dataset))
train_size = len(dataset) - test_size

# Use random_split() to generate train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing  graphs: {len(test_dataset)}')

Number of training graphs: 73670
Number of testing  graphs: 0


# Training of the model

In [8]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = nGCNN(n_features, dropout_node).to(device)
edge_model = eGCNN(n_features, dropout_edge).to(device)

# Load previous model if available
try:
    node_model.load_state_dict(torch.load(node_model_name))
    edge_model.load_state_dict(torch.load(edge_model_name))
except FileNotFoundError:
    pass

# Evaluate model state
node_model.eval()
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


In [15]:
import itertools
from pymatgen.core.structure import Composition, Element, PeriodicSite, Structure
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import fcluster, linkage
from pymatgen.analysis.structure_matcher import StructureMatcher

In [17]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt

In [45]:
b[j]['site']

PeriodicNeighbor: I (3.915, -0.5117, 10.28) [0.3104, -0.02982, 1.0]

In [47]:
from pymatgen.analysis.local_env import VoronoiNN
voronoi = VoronoiNN()

# Load the structure to be relaxed
structure = Structure.from_file('POSCAR')

neighbors_list = []
for i, site in enumerate(structure):
    voronoi_polyhedra_i = voronoi.get_voronoi_polyhedra(structure, i)
    neighbors_list_i = []
    for j in voronoi_polyhedra_i:
        neighbors_list_i.append(voronoi_polyhedra_i[j]['site'])
    neighbors_list.append(neighbors_list_i)
neighbors_list

[[PeriodicNeighbor: S (-1.317, 11.59, -3.746) [-0.1044, 0.6754, -0.3643],
  PeriodicNeighbor: S (3.121, 13.33, -3.284) [0.2475, 0.7767, -0.3194],
  PeriodicNeighbor: Bi (1.73e-06, 17.16, -2.53e-06) [0.0, 1.0, 0.0],
  PeriodicNeighbor: I (-2.762, 14.0, -1.303) [-0.219, 0.816, -0.1267],
  PeriodicNeighbor: Bi (5.238, 12.16, 3.865) [0.4153, 0.7089, 0.3759],
  PeriodicNeighbor: I (3.915, 16.65, -4.349e-16) [0.3104, 0.9702, -1.117e-06],
  PeriodicNeighbor: S (4.65, 13.12, 1.344) [0.3687, 0.7645, 0.1307],
  PeriodicNeighbor: S (2.26, 10.72, 0.2364) [0.1792, 0.6249, 0.023],
  PeriodicNeighbor: I (3.331, 15.54, 3.89) [0.2641, 0.9058, 0.3783],
  PeriodicNeighbor: Bi (-2.469, 14.8, 1.913) [-0.1958, 0.8626, 0.186],
  PeriodicNeighbor: Bi (0.7458, 14.06, 4.447) [0.05914, 0.8194, 0.4325],
  PeriodicNeighbor: S (0.9144, 17.16, -4.743e-05) [0.07251, 1.0, -4.683e-06],
  PeriodicNeighbor: S (-0.1677, 14.04, 4.484) [-0.0133, 0.8185, 0.4361],
  PeriodicNeighbor: I (-1.163, 11.27, 2.795) [-0.09224, 0.6565

In [49]:
cell_range = list(range(-1, 2))
for shift in itertools.product(cell_range, cell_range, cell_range):
    for site in prim_structure.sites:
        shifted = site.frac_coords + shift
        print(site.frac_coords, shift)

[0.15303071 0.75673391 0.13993897] (-1, -1, -1)
[0.04850789 0.33747808 0.21009979] (-1, -1, -1)
[0.78817676 0.38314731 0.03723586] (-1, -1, -1)
[0.80421219 0.86255861 0.18604014] (-1, -1, -1)
[0.531004   0.42821624 0.28469831] (-1, -1, -1)
[0.54711752 0.90886171 0.05523736] (-1, -1, -1)
[0.51635022 0.91450422 0.49482602] (-1, -1, -1)
[0.53354546 0.39700903 0.71387018] (-1, -1, -1)
[0.2590788  0.95846747 0.24751013] (-1, -1, -1)
[0.27551075 0.44315056 0.46361096] (-1, -1, -1)
[0. 0. 0.] (-1, -1, -1)
[0.01896755 0.48553628 0.21323449] (-1, -1, -1)
[0.05913883 0.8194385  0.43254868] (-1, -1, -1)
[0.91534096 0.54435297 0.51965487] (-1, -1, -1)
[0.64877958 0.11154693 0.04763532] (-1, -1, -1)
[0.65792344 0.59368077 0.26053094] (-1, -1, -1)
[0.38998341 0.15602175 0.20007815] (-1, -1, -1)
[0.38860894 0.63984714 0.00730784] (-1, -1, -1)
[0.65669485 0.18352299 0.41087729] (-1, -1, -1)
[0.6715983  0.6663373  0.62907822] (-1, -1, -1)
[0.39768375 0.22869507 0.1636588 ] (-1, -1, -1)
[0.41533396 0.70

In [54]:
structure = Structure.from_sites(structure, to_unit_cell=True)
prim_structure = structure.get_primitive_structure()
# Voronoi polyhedra can extend beyond the standard unit cell
coords = []
cell_range = list(range(-1, 2))
for shift in itertools.product(cell_range, cell_range, cell_range):
    for site in prim_structure.sites:
        shifted = site.frac_coords + shift
        coords.append(prim_structure.lattice.get_cartesian_coords(shifted))

In [62]:
"""
Get the Voronoi nodes of a structure.
Templated from the TopographyAnalyzer class, added to
pymatgen.analysis.defects.utils by Yiming Chen, but now deleted.
Modified to map down to primitive, do Voronoi analysis, then map
back to original supercell; much more efficient.

Args:
    structure (:obj:`Structure`):
        pymatgen `Structure` object.
""" 

# Map all sites to the unit cell; 0 ≤ xyz < 1
structure = Structure.from_sites(structure, to_unit_cell=True)

# Get Voronoi nodes in primitive structure and then map back to the
# supercell
prim_structure = structure.get_primitive_structure()

# Get all atom coords in a supercell of the structure because
# Voronoi polyhedra can extend beyond the standard unit cell
coords = []
cell_range = list(range(-1, 2))
for shift in itertools.product(cell_range, cell_range, cell_range):
    for site in prim_structure.sites:
        shifted = site.frac_coords + shift
        coords.append(prim_structure.lattice.get_cartesian_coords(shifted))

# Voronoi tessellation
voro = Voronoi(coords)

# Only include voronoi polyhedra within the unit cell
vnodes = []
tol = 1e-3
for vertex in voro.vertices:
    frac_coords = prim_structure.lattice.get_fractional_coords(vertex)
    vnode = PeriodicSite("V-", frac_coords, prim_structure.lattice)
    if np.all([-tol <= coord < 1 + tol for coord in frac_coords]):
        # vnode within unit cell
        
        if all(p.distance(vnode) >= tol for p in vnodes):
            vnodes.append(vnode)

In [74]:
voro.ridge_points

array([[  60,   70],
       [  60,   18],
       [  60,   44],
       ...,
       [1035,  789],
       [ 824, 1039],
       [1039, 1062]], dtype=int32)

In [18]:
# Cluster nodes that are within a certain distance of each other
voronoi_coords = [v.frac_coords for v in vnodes]
dist_matrix = np.array(
    prim_structure.lattice.get_all_distances(voronoi_coords, voronoi_coords)
)
dist_matrix = (dist_matrix + dist_matrix.T) / 2
condensed_m = squareform(dist_matrix)
z = linkage(condensed_m)
cn = fcluster(z, 0.2, criterion="distance")  # cluster nodes with 0.2 Å
merged_vnodes = []
for n in set(cn):
    frac_coords = []
    for i, j in enumerate(np.where(cn == n)[0]):
        if i == 0:
            frac_coords.append(vnodes[j].frac_coords)
        else:
            fcoords = vnodes[j].frac_coords
            d, image = prim_structure.lattice.get_distance_and_image(
                frac_coords[0], fcoords
            )
            frac_coords.append(fcoords + image)
    merged_vnodes.append(
        PeriodicSite("V-", np.average(frac_coords, axis=0), prim_structure.lattice)
    )
vnodes = merged_vnodes

# Remove nodes less than 0.5 Å from sites in the structure
vfcoords = [v.frac_coords for v in vnodes]
sfcoords = prim_structure.frac_coords
dist_matrix = prim_structure.lattice.get_all_distances(vfcoords, sfcoords)
all_dist = np.min(dist_matrix, axis=1)
vnodes = [v for i, v in enumerate(vnodes) if all_dist[i] > 0.5]

# Map back to the supercell
sm = StructureMatcher(primitive_cell=False, attempt_supercell=True)
mapping = sm.get_supercell_matrix(structure, prim_structure)
voronoi_struc = Structure.from_sites(
    vnodes
)  # Structure object with Voronoi nodes as sites
voronoi_struc.make_supercell(mapping)  # Map back to the supercell

# Check if there was an origin shift between primitive and supercell
regenerated_supercell = prim_structure.copy()
regenerated_supercell.make_supercell(mapping)
fractional_shift = sm.get_transformation(structure, regenerated_supercell)[1]
if not np.allclose(fractional_shift, 0):
    voronoi_struc.translate_sites(
        range(len(voronoi_struc)), fractional_shift, frac_coords=True
    )

vnodes = voronoi_struc.sites
vnodes

[PeriodicSite: V- (11.44, 7.346, 1.32) [0.9071, 0.4281, 0.1284],
 PeriodicSite: V- (11.28, 7.622, 1.314) [0.8942, 0.4442, 0.1278],
 PeriodicSite: V- (12.58, 6.261, 5.353) [0.9974, 0.3649, 0.5206],
 PeriodicSite: V- (0.1271, 6.436, 5.199) [0.01008, 0.3751, 0.5056],
 PeriodicSite: V- (12.53, 5.925, 5.316) [0.9936, 0.3453, 0.5171],
 PeriodicSite: V- (0.09451, 5.558, 4.988) [0.007492, 0.3239, 0.4851],
 PeriodicSite: V- (12.13, 6.399, 5.096) [0.962, 0.3729, 0.4956],
 PeriodicSite: V- (11.85, 5.941, 4.921) [0.9398, 0.3462, 0.4786],
 PeriodicSite: V- (0.5996, 6.954, 4.87) [0.04755, 0.4053, 0.4737],
 PeriodicSite: V- (0.4489, 7.032, 4.703) [0.0356, 0.4098, 0.4574],
 PeriodicSite: V- (0.1594, 6.991, 4.597) [0.01264, 0.4074, 0.447],
 PeriodicSite: V- (12.25, 7.11, 4.343) [0.9715, 0.4144, 0.4224],
 PeriodicSite: V- (10.89, 8.061, 7.878) [0.8632, 0.4698, 0.7662],
 PeriodicSite: V- (10.44, 7.697, 8.285) [0.8281, 0.4486, 0.8058],
 PeriodicSite: V- (10.24, 8.218, 7.39) [0.812, 0.479, 0.7187],
 Period

In [ ]:
# Collect connected node pairs
connected_node_pairs = set()
for region in voro.regions:
    if not -1 in region and len(region) > 0:
        face = [vnodes[i] for i in region]
        indices = [vnodes.index(site) for site in face]
        indices = sorted(set(indices))
        if len(indices) == 2:
            connected_node_pairs.add(tuple(indices))

# Print or use the connected node pairs
print(connected_node_pairs)

In [56]:
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import seaborn           as sns

from os import path

In [75]:
vnodes_c = np.array([vnode.coords.tolist() for vnode in vnodes])

In [76]:
vnodes_c

array([[11.43934546,  7.34582549,  1.32016287],
       [11.2766999 ,  7.62204093,  1.3136017 ],
       [12.57780065,  6.26102123,  5.3528403 ],
       ...,
       [10.34337643,  7.32106441,  5.07801331],
       [ 3.84719231, 11.60821415,  5.62896012],
       [10.90669207, 16.64705145,  7.73146986]])

In [80]:
%matplotlib qt

In [82]:
ax = plt.axes(projection='3d')
footnotesize = 10
ax.set_xlabel(r'x ($\AA$)', fontsize=footnotesize)
ax.set_ylabel(r'y ($\AA$)', fontsize=footnotesize)
ax.set_zlabel(r'z ($\AA$)', fontsize=footnotesize)

ax.tick_params(axis='x', labelsize=footnotesize)
ax.tick_params(axis='y', labelsize=footnotesize)
ax.tick_params(axis='z', labelsize=footnotesize)

ax.scatter(vnodes_c[:, 0],
           vnodes_c[:, 1],
           vnodes_c[:, 2],
           marker='.',
           c='b')

ax.scatter(structure.cart_coords[:, 0],
           structure.cart_coords[:, 1],
           structure.cart_coords[:, 2],
           marker='o',
           c='r')

plt.show()

In [8]:
# Initialize the optimizers
node_optimizer = torch.optim.Adam(node_model.parameters(), lr=learning_rate)
edge_optimizer = torch.optim.Adam(edge_model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in range(n_epochs):
    # Initialize train loss variable
    train_loss = 0
    for graph in train_dataset:
        #print()
        # Clone existing graph
        graph_0 = graph.clone()
        
        # Initialize the gradient of the optimizers
        node_optimizer.zero_grad()
        edge_optimizer.zero_grad()
        
        # Start denoising-diffusing process
        for t_step in np.arange(1, n_t_steps+1):
            # Diffuse the graph with some noise
            #print()
            #print(f'Step: {t_step}')
            #print('Diffusing...')
            
            graph_t, epsilon_t = diffusion_step(graph_0, t_step, n_t_steps, alpha_decay)
            
            # Update diffused graph as next one
            graph_0 = graph_t.clone()

            # Denoise the diffused graph
            #print(f'Denoising...')
            
            # Add t_step information to graph_t
            t_step_std = (t_step/n_t_steps - 0.5)  # Standard normalization
            graph_t = add_features_to_graph(graph_t,
                                                torch.tensor([[t_step_std]]))  # To match graph.y shape
            
            # Add target information
            #graph_t = add_features_to_graph(graph_t,
            #                                    graph_t.y)

            # Perform a single forward pass for predicting node features
            out_x = node_model(graph_t.x,
                               graph_t.edge_index,
                               graph_t.edge_attr)
            
            # Remove t_step information
            out_x = out_x[:, :-1]

            # Define x_i and x_j as features of every corresponding pair of nodes (same order than attributes)
            x_i = graph_t.x[graph_t.edge_index[0]]
            x_j = graph_t.x[graph_t.edge_index[1]]

            # Perform a single forward pass for predicting edge attributes
            # Introduce previous edge attributes as features as well
            out_attr = edge_model(x_i, x_j, graph_t.edge_attr)

            # Construct noise graph
            pred_epsilon_t = Data(x=out_x,
                                  edge_index=graph_t.edge_index,
                                  edge_attr=out_attr.ravel())
            
            # Backpropagation and optimization step
            #print('Backpropagating...')

            # Calculate the loss for node features and edge attributes
            node_loss, edge_loss = get_graph_losses(epsilon_t, pred_epsilon_t)
            
            # Backpropagate and optimize node loss
            node_loss.backward(retain_graph=True)
            node_optimizer.step()

            # Backpropagate and optimize edge loss
            edge_loss.backward(retain_graph=True)
            edge_optimizer.step()

            # Accumulate the total training loss
            loss = node_loss + edge_loss
            train_loss += loss.item()
    
    # Compute the average train loss
    train_loss = train_loss / (len(train_dataset) * n_t_steps)
    train_losses.append(train_loss)
    
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')
    
    # Save some checkpoints
    if (epoch % 20) == 0:
        torch.save(node_model.state_dict(), node_model_name)
        torch.save(edge_model.state_dict(), edge_model_name)

torch.save(node_model.state_dict(), node_model_name)
torch.save(edge_model.state_dict(), edge_model_name)

Epoch: 1, Train Loss: 2.0373
Epoch: 2, Train Loss: 1.9311
Epoch: 3, Train Loss: 1.7960
Epoch: 4, Train Loss: 1.7212
Epoch: 5, Train Loss: 1.6750
Epoch: 6, Train Loss: 1.6145
Epoch: 7, Train Loss: 1.5966
Epoch: 8, Train Loss: 1.5818
Epoch: 9, Train Loss: 1.5717
Epoch: 10, Train Loss: 1.5618
Epoch: 11, Train Loss: 1.5611
Epoch: 12, Train Loss: 1.5478
Epoch: 13, Train Loss: 1.5490
Epoch: 14, Train Loss: 1.5486
Epoch: 15, Train Loss: 1.5475
Epoch: 16, Train Loss: 1.5475
Epoch: 17, Train Loss: 1.5417
Epoch: 18, Train Loss: 1.5389
Epoch: 19, Train Loss: 1.5384
Epoch: 20, Train Loss: 1.5386
Epoch: 21, Train Loss: 1.5360
Epoch: 22, Train Loss: 1.5378
Epoch: 23, Train Loss: 1.5377
Epoch: 24, Train Loss: 1.5463
Epoch: 25, Train Loss: 1.5320
Epoch: 26, Train Loss: 1.5332
Epoch: 27, Train Loss: 1.5292
Epoch: 28, Train Loss: 1.5302
Epoch: 29, Train Loss: 1.5321
Epoch: 30, Train Loss: 1.5313
Epoch: 31, Train Loss: 1.5281
Epoch: 32, Train Loss: 1.5275
Epoch: 33, Train Loss: 1.5286
Epoch: 34, Train Lo

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.log(train_losses))
plt.xlabel('Epoch')
plt.ylabel('Loss function')